This is evaluation for トキポナ1000文コーパス by ponaLM

PonaLM is still weak to evaluate correctly. So the fluency score might be wrong.

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import os, sys, re, regex
sys.path.append("../../../ponaLM")
from models import PonaLM
pd.set_option("display.max_colwidth", 1000)

prepare toki pona language model and dictionary

In [2]:
with open('../../words.txt') as f:
    words = [word.rstrip() for word in f.readlines()]
    w2n = {w:i for i,w in enumerate(words)}

# for detect proper noun
def proper(word):
    return re.match(r'^([AIUEO]|[KSNPML][aiueo]|[TJ][aueo]|W[aie])n?(([ksnpml]?[aiueo]|[tj][aueo]|w[aie])n?)*$',word)

# for detect symbols
p_emoji = regex.compile(r'\p{Emoji=Yes}+')

def word_to_id(word):
    if word in w2n:
        return w2n[word]
    elif proper(word):
        return w2n['<Q>']
    elif word.isnumeric():
        return w2n['<NUM>']
    elif p_emoji.search(word):
        return w2n['<SYM>']
    else:
        assert mopu

In [3]:
model = PonaLM(len(words))
model.load_state_dict(torch.load('../../../ponaLM/lm.model', map_location='cpu'))
model.eval()

PonaLM(
  (embed): Embedding(145, 20)
  (lstmf): LSTM(20, 100)
  (lstmb): LSTM(20, 100)
  (h2h): Linear(in_features=100, out_features=20, bias=True)
  (h2o): Linear(in_features=20, out_features=145, bias=True)
  (drop1): Dropout(p=0.03)
  (drop2): Dropout(p=0.03)
  (drop3): Dropout(p=0.2)
  (drop4): Dropout(p=0.2)
)

code for evaluation

In [4]:
# Negotive Log Likelihhod
def NLL(sent):
    sent = [word_to_id(w) for w in sent]
    pred = model(torch.tensor([[0,0] + sent]), torch.tensor([[0,0] + sent[::-1]]))
    return [-p[w].item() for p,w in zip(pred[1:-1], sent)]

# evaluate the score of sentences
def evaluate(sent): 
    nlls = NLL(sent)
    prob = [round(np.exp(-l)*100,2) for l in nlls] # probability = exp(-NLL)
    fluency = round(1/(1+np.mean(nlls)),3)         # fluency = 1 / (1 + ave(all NLL))
    display(pd.DataFrame([prob + [fluency]], index=['prob'], columns=sent + ['Fluency']))

load corpus

In [5]:
sents = [sent.rstrip().split(' ') for sent in open('pona_corpus1000.txt')]
sents[0]

['mi', 'jan', 'Alito', '.']

evaluate fluency of each sentence

In [6]:
for i, line in enumerate(sents):
    print('#{:04}'.format(i+1))
    evaluate(line)

#0001


,mi,jan,Alito,.,Fluency
prob,1.15,43.04,66.25,89.63,0.407


#0002


,ali,li,ike,tawa,ona,.,Fluency
prob,0.42,80.63,3.75,39.23,8.12,89.73,0.324


#0003


...

,jan,Ototake,li,jo,ala,e,luka,en,noka,.,Fluency
prob,76.84,70.5,95.01,4.7,63.14,58.53,0.32,1.31,0.29,89.42,0.325


#0059


,tomo,tawa,pi,sitelen,tawa,ni,li,ilo,tawa,tenpo,.,Fluency
prob,0.89,3.38,9.68,1.39,6.93,15.3,19.51,0.22,20.17,0.19,90.62,0.239


#0060


,mi,sona,ala,e,nimi,sina,.,Fluency
prob,40.86,5.66,61.53,80.02,3.88,10.96,78.68,0.408
